In [5]:
import pandas as pd
from os import chdir
from sklearn.ensemble import RandomForestClassifier

In [10]:
chdir("../Packages/") 
import pymorphy2

In [11]:
chdir("../Compile")
import Data_filter as DF
import Analyzier

In [12]:
TrainingData = pd.DataFrame.from_csv("../../Data/training_data.csv")

In [13]:
Data = TrainingData['sentence'].apply(DF.ParseSentence)
Data = Data.to_frame()

In [14]:
Data['?'] = Data['sentence'].apply(lambda x: x[1]['?'])
Data['!'] = Data['sentence'].apply(lambda x: x[1]['!'])
Data['.'] = Data['sentence'].apply(lambda x: x[1]['.'])
Data['sum'] = Data['sentence'].apply(lambda x: Analyzier.AnalyzeWords(x[0]))
del(Data['sentence'])

In [15]:
Data.head(3)

,?,!,.,sum
1,0,1,0,-3.0
2,1,0,0,0.0
3,1,0,0,0.5


In [16]:
Training = TrainingData['label'].values

In [17]:
model =  RandomForestClassifier(n_estimators=1000)

In [18]:
from sklearn.cross_validation import train_test_split
from sklearn.metrics import f1_score
Xtr,Xval,Ytr,Yval = train_test_split(Data,Training,test_size=0.3,random_state=128)

/root/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [19]:
model.fit(Xtr, Ytr)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=1000, n_jobs=1, oob_score=False,
            random_state=None, verbose=0, warm_start=False)

In [20]:
print(f1_score(Yval, model.predict(Xval), average='macro'))

0.420518314458


In [28]:
Input = open("../../Data/data.txt")
PredictionData = []
for line in Input.readlines():
    PredictionData.append(line)
del(PredictionData[0])
Input.close()

In [29]:
PredictionData = pd.DataFrame(PredictionData)
PredictionData.columns = ['sentence']

In [30]:
PredictionData['parsed'] = PredictionData['sentence'].apply(DF.ParseSentence)
del(PredictionData['sentence'])

In [31]:
PredictionData['?'] = PredictionData['parsed'].apply(lambda x: x[1]['?'])
PredictionData['!'] = PredictionData['parsed'].apply(lambda x: x[1]['!'])
PredictionData['.'] = PredictionData['parsed'].apply(lambda x: x[1]['.'])
PredictionData['sum'] = PredictionData['parsed'].apply(lambda x: Analyzier.AnalyzeWords(x[0]))
del(PredictionData['parsed'])

In [32]:
Answer = model.predict(PredictionData)

In [33]:
Output = open('result.txt', 'w')
Output.write("\"label\""+"\n")

8

In [34]:
for i in Answer:
    Output.write("\""+str(i)+"\""+"\n")
Output.close()